In [ ]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

###  Carga de datos

In [ ]:
auctions_18_20 = auctions_19_21 = auctions_20_22 = auctions_21_23 = None
installs_18_20 = installs_19_21 = installs_20_22 = installs_21_23 = None
events_18_20 = events_19_21 = events_20_22 = events_21_23 = None
clicks_18_20 = clicks_19_21 = clicks_20_22 = clicks_21_23 = None

In [ ]:
auctions_18_20 = pd.read_csv('windows/18_20/auctions.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)
auctions_19_21 = pd.read_csv('windows/19_21/auctions.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)
auctions_20_22 = pd.read_csv('windows/20_22/auctions.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)
auctions_21_23 = pd.read_csv('windows/21_23/auctions.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)

In [ ]:
installs_18_20 = pd.read_csv('windows/18_20/installs.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)
installs_19_21 = pd.read_csv('windows/19_21/installs.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)
installs_20_22 = pd.read_csv('windows/20_22/installs.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)
installs_21_23 = pd.read_csv('windows/21_23/installs.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)

In [ ]:
clicks_18_20 = pd.read_csv('windows/18_20/clicks.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)
clicks_19_21 = pd.read_csv('windows/19_21/clicks.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)
clicks_20_22 = pd.read_csv('windows/20_22/clicks.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)
clicks_21_23 = pd.read_csv('windows/21_23/clicks.csv', parse_dates = ['created'], infer_datetime_format = True, low_memory=False)


In [ ]:
events_18_20 = pd.read_csv('windows/18_20/events.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)
events_19_21 = pd.read_csv('windows/19_21/events.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)
events_20_22 = pd.read_csv('windows/20_22/events.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)
events_21_23 = pd.read_csv('windows/21_23/events.csv', parse_dates = ['date'], infer_datetime_format = True, low_memory=False)

In [ ]:
df_features_18_20 = pd.read_csv('windows/18_20/features.csv', low_memory=False)
df_features_18_20.set_index('ref_hash', inplace=True)

In [ ]:
df_features_19_21 = pd.read_csv('windows/19_21/features.csv', low_memory=False)
df_features_19_21.set_index('ref_hash', inplace=True)

In [ ]:
df_features_20_22 = pd.read_csv('windows/20_22/features.csv', low_memory=False)
df_features_20_22.set_index('ref_hash', inplace=True)

In [ ]:
df_features_21_23 = pd.read_csv('windows/21_23/features.csv', low_memory=False)
df_features_21_23.set_index('ref_hash', inplace=True)

### Partición de los datos por ventanas

In [ ]:
ventanas = [
    {
        "auctions": auctions_18_20,
        "installs": installs_18_20,
        "clicks": clicks_18_20,
        "events": events_18_20,
        "features": df_features_18_20,
        "name": "18_20"
    },
    {
        "auctions": auctions_19_21,
        "installs": installs_19_21,
        "clicks": clicks_19_21,
        "events": events_19_21,
        "features": df_features_19_21,        
        "name": "19_21"
    },
    {
        "auctions": auctions_20_22,
        "installs": installs_20_22,
        "clicks": clicks_20_22,
        "events": events_20_22,
        "features": df_features_20_22,        
        "name": "20_22"
    },
    {
        "auctions": auctions_21_23,
        "installs": installs_21_23,
        "clicks": clicks_21_23,
        "events": events_21_23,
        "features": df_features_21_23,        
        "name": "21_23"
    }
]

### Funcion para agregar features

In [ ]:
def add_feature(df_features, df_to_join, feature_name, fill = 0):
    df_features = df_features.merge(df_to_join[[feature_name]], how = 'left', left_index = True, right_index = True).\
    fillna(fill)
    return df_features

In [ ]:
def agregar_features(ventanas, feature_function, feature_name):
    for ventana in ventanas:
        auctions = ventana["auctions"]
        installs = ventana["installs"]
        events = ventana["events"]
        clicks = ventana["clicks"]
        feature = ventana["features"]
        
        df_to_join = feature_function(auctions, installs, events, clicks)
        ventana["features"] = add_feature(feature, df_to_join, feature_name)
    save_changes(ventanas)
    return ventanas

In [ ]:
def save_changes(ventanas):
    for ventana in ventanas:
        feature = ventana["features"]
        name = ventana["name"]
        feature.to_csv("windows/{}/features.csv".format(name), index = True)

## Features

In [ ]:
def user_appeared_last_day(auctions, installs, events, clicks):
    ultimo = auctions["date"].max().day
    auc_feature2 = auctions.loc[auctions["date"].dt.day == ultimo]
    auc_feature2['user_appeared_last_day'] = 1
    auc_feature2 = auc_feature2.groupby('ref_hash').agg({'user_appeared_last_day' : 'first'})
    return auc_feature2

In [ ]:
ventantas = agregar_features(ventanas, user_appeared_last_day, "user_appeared_last_day")

> # Sección para añadir features

> *¡Atención! Antes de usar la función add_feature, setear como indice el id de los dispositivos en el df que contiene el feature que se desea agregar*

> ### ***Feature 1: cantidad de veces que el usuario aparece en una subasta***

In [ ]:
auctions_18_20['appearances_in_auctions'] = 1
auc_feature1 = auctions_18_20.groupby('device_id').agg({'appearances_in_auctions': 'count'})
auc_18_20_features = add_feature(auc_18_20_features, auc_feature1, "appearances_in_auctions")

In [ ]:
auc_18_20_features.head(1)

> ### ***Feature 2: ¿apareció en una subasta el último día?***

In [ ]:
auc_feature2 = auctions_18_20.loc[auctions_18_20["date"].dt.day == 20]
auc_feature2['user_appeared_last_day'] = 1
auc_feature2 = auc_feature2.groupby('device_id').agg({'user_appeared_last_day' : 'first'})
auc_18_20_features = add_feature(auc_18_20_features, auc_feature2, "user_appeared_last_day")

> ### ***Feature 3: en promedio, ¿ cuánto tardo el usuario en reaparecer en una subasta?***

In [ ]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby(feature_id_name).agg({'time_to_reappear': 'mean'})
    return df_sorted

In [ ]:
auc_feature3 = calculate_time_to_reappear(auctions_18_20, "date", "device_id")


auc_18_20_features = add_feature(auc_18_20_features, auc_feature3, "time_to_reappear")

> ### Feature 4: Día de la semana, día del mes, hora del día, segundo de la hora, minuto de la hora

In [ ]:
auc_feature4 = auctions_18_20.copy()

# Día de la semana
auc_feature4["dayofweek"] = auc_feature4.date.dt.dayofweek

# Día del mes
auc_feature4["day"] = auc_feature4.date.dt.day

# Hora del día
auc_feature4["hour"] = auc_feature4.date.dt.hour

# Segundo de la hora
auc_feature4["second"] = auc_feature4.date.dt.second

# Minuto de la hora
auc_feature4["minute"] = auc_feature4.date.dt.minute


auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "dayofweek")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "day")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "second")
auc_18_20_features = add_feature(auc_18_20_features, auc_feature4, "minute")

> ### Feature 5: Separación del día en 4 partes

In [ ]:
def divide_time_day(df):
    df.loc[(0 <= df["hour"]) & (df["hour"] < 6), 'timeofday'] = 0
    df.loc[(6 <= df["hour"]) & (df["hour"] < 13), 'timeofday'] = 1
    df.loc[(13 <= df["hour"]) & (df["hour"] < 19), 'timeofday'] = 2
    df.loc[(19 <= df["hour"]) & (df["hour"] < 24), 'timeofday'] = 3
    df["timeofday"] = df["timeofday"].astype(int)
    return df

auc_feature5 = auctions_18_20.copy()
auc_feature5["hour"] = auc_feature5.date.dt.hour
auc_feature5 = divide_time_day(auc_feature5)
auc_18_20_features = add_feature(auc_18_20_features, auc_feature5, "timeofday")


> ### Feature 6: Días feriados en Uruguay

In [ ]:
# En Uruguay hubo 3 días feriados en el rango de fecha que nos dieron
# El 18 y 19 de Abril fueron Jueves y Viernes Santo
# El 22 fue el día de Desembarco de los 33 orientales

def holidays_days(df):
    df.loc[(22 == df["day"]) | (19 == df["day"]) | (18 == df["day"]), 'holidaysday'] = 1
    return df


auc_feature6 = auctions_18_20.copy()
auc_feature6["day"] = auc_feature5.date.dt.day
auc_feature6 = holidays_days(auc_feature6)
auc_feature6 = auc_feature6.fillna(0)

> ### Feature 7: Días lluviosos en Uruguay

In [ ]:
# Días que llovieron del 18 al 26: El 26 fue el único día que llovió
def raining_days(df):
    df.loc[26 == df["day"], 'rainingdays'] = 1
    return df

auc_feature7 = auctions_18_20.copy()
auc_feature7["day"] = auc_feature7.date.dt.day
auc_feature7 = raining_days(auc_feature7)
auc_feature7 = auc_feature7.fillna(0)


> ### ***Feature 1: cantidad de clicks realizados por el usuario***

In [ ]:
clicks_18_20['amount_of_clicks'] = 1
inst_feature1 = clicks_18_20.groupby('ref_hash').agg({'amount_of_clicks': 'count'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature1, "amount_of_clicks")

In [ ]:
def cant_clicks(auctions, installs, events, clicks):
    clicks["amount_of_clicks"] = 1
    clicks_feature = clicks.groupby('ref_hash').agg({'amount_of_clicks': 'count'})
    return clicks_feature

In [ ]:
ventanas = agregar_features(ventanas, cant_clicks, 'amount_of_clicks')

> ### ***Feature 2: ¿realizó una instalación?***

In [ ]:
installs_18_20['has_installed'] = 1
inst_feature2 = installs_18_20.groupby('ref_hash').agg({'has_installed': 'first'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature2, "has_installed")

In [ ]:
installs_19_21['has_installed'] = 1
inst_feature2 = installs_19_21.groupby('ref_hash').agg({'has_installed': 'first'})
inst_19_21_features = add_feature(inst_19_21_features, inst_feature2, "has_installed")

> ### ***Feature 3: ¿realizó un click el último día?***

In [ ]:
inst_feature3 = clicks_18_20.loc[clicks_18_20["created"].dt.day == 20]
inst_feature3['user_clicked_last_day'] = 1
inst_feature3 = inst_feature3.groupby('ref_hash').agg({'user_clicked_last_day' : 'first'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature3, "user_clicked_last_day")

> ### ***Feature 4: ¿realizó una instalación el último día?***

In [ ]:
inst_feature4 = installs_18_20.loc[installs_18_20["created"].dt.day == 20]
inst_feature4['user_installed_last_day'] = 1
inst_feature4 = inst_feature4.groupby('ref_hash').agg({'user_installed_last_day' : 'first'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature4, "user_installed_last_day")

> ### ***Feature 5: cantidad de instalaciones realizadas***

In [ ]:
installs_18_20['amount_of_installs'] = 1
inst_feature5 = installs_18_20.groupby('ref_hash').agg({'amount_of_installs': 'count'})
inst_18_20_features = add_feature(inst_18_20_features, inst_feature5, "amount_of_installs")

> ### ***Feature 6: Mínimo de clicks en un día***

In [ ]:
clicks_18_20["day"] = clicks_18_20.created.dt.day
clicks_feature5 = clicks_18_20.groupby(["ref_hash", "day"]).agg({"cantidad":"sum"})
clicks_feature5 = pd.DataFrame(clicks_feature5.reset_index().groupby('ref_hash')['cantidad'].max()).reset_index()
clicks_feature5 = clicks_feature5.rename(columns={"cantidad":"cant_max_day"})
clicks_18_20_features = add_feature(clicks_18_20_features, clicks_feature5, "cant_max_day")


> ### Feature 6: Máximo de clicks en un día

In [ ]:
clicks_feature6 = clicks_18_20.groupby(["ref_hash", "day"]).agg({"cantidad":"sum"})
clicks_feature6 = pd.DataFrame(clicks_feature5.reset_index().groupby('ref_hash')['cantidad'].min()).reset_index()
clicks_feature6 = clicks_feature5.rename(columns={"cantidad":"cant_min_day"})

> ### Feature 7: Promedio de tiempo entre clicks

In [ ]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name, function_type):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby(feature_id_name).agg({'time_to_reappear': function_type})
    return df_sorted

clicks_feature7 = calculate_time_to_reappear(clicks_18_20, "created", "ref_hash", "mean")
clicks_18_20_features = add_feature(clicks_18_20_features, clicks_feature5, "time_to_reappear")

> ### Feature 8: Tiempo máximo entre clicks

In [ ]:
clicks_feature8 = calculate_time_to_reappear(clicks_18_20, "created", "ref_hash", "mean")

> ### Feature 9: Tiempo mínimo entre clicks

In [ ]:
clicks_feature9 = calculate_time_to_reappear(clicks_18_20, "created", "ref_hash", "mean")